In [7]:
### Library Imports

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
import time


In [15]:
### Perform Scrape

# Set up Chrome WebDriver
driver = webdriver.Chrome(executable_path="/Users/mburley/chromedriver/chromedriver-mac-x64/chromedriver")  # Replace with the path to your chromedriver

# Open the Oakley Forum login page
login_url = "https://www.oakleyforum.com/login/"
driver.get(login_url)

# Simulate login (replace 'your_username' and 'your_password' with your actual credentials)
try:
    # Look for the login form by waiting for it to be present
    login_form = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//form[contains(@action, "/login/login")]')))
except NoSuchElementException:
    print("Login form not found. Check the HTML structure.")

# Input credentials and submit the form
username_input = login_form.find_element(By.XPATH, '//input[contains(@name, "login")]')
password_input = login_form.find_element(By.XPATH, '//input[contains(@name, "password")]')

username_input.send_keys('bandit.sunglasses@gmail.com')
password_input.send_keys('OakleyIsCool1!')

login_form.submit()

# Wait for login to complete
wait = WebDriverWait(driver, 10)
try:
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "structItem-cell")))
except TimeoutException:
    print("Timed out waiting for page to load after login.")

data = []
current_page = 1

while current_page <= 3:
    # Get the page source after waiting for dynamic content
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html5lib')

    print(f"Extracting page: {driver.current_url}")

    for listing in soup.select(".structItem-cell"):
        # Title of the listing
        title_element = listing.select_one(".structItem-title")
        title = title_element.text.strip() if title_element else None

        # Selling button
        selling_button_element = listing.select_one("labelLink")
        selling_button = selling_button_element.text.strip() if selling_button_element else None

        # Date
        date_element = listing.select_one(".structItem-startDate")
        date = date_element.text.strip() if date_element else None

        # Username
        username_element = listing.select_one(".username")
        username = username_element.text.strip() if username_element else None

        data.append({
            "title": title,
            "selling_button": selling_button,
            "date": date,
            "username": username
        })

    try:
        # Construct the URL for the next page
        next_page_url = f"https://www.oakleyforum.com/forums/oakley-sunglasses-exchange.8/page-{current_page + 1}?prefix_id=2"

        # Navigate to the next page
        driver.get(next_page_url)

        # Wait for the new page to load
        wait.until(EC.url_contains(f"page-{current_page + 1}"))

        # Increment current_page for the next iteration
        current_page += 1

    except TimeoutException:
        # Break if unable to navigate to the next page
        break

# Save to CSV
df = pd.DataFrame(data=data)
df.to_csv("oakley_forum_data.csv", index=False)

# Close the WebDriver
driver.quit()

print(df)

Timed out waiting for page to load after login.
Extracting page: https://www.oakleyforum.com/
Extracting page: https://www.oakleyforum.com/forums/oakley-sunglasses-exchange.8/page-2?prefix_id=2
Extracting page: https://www.oakleyforum.com/forums/oakley-sunglasses-exchange.8/page-3?prefix_id=2
Extracting page: https://www.oakleyforum.com/forums/oakley-sunglasses-exchange.8/page-4?prefix_id=2
Extracting page: https://www.oakleyforum.com/forums/oakley-sunglasses-exchange.8/page-5?prefix_id=2
Extracting page: https://www.oakleyforum.com/forums/oakley-sunglasses-exchange.8/page-6?prefix_id=2


KeyboardInterrupt: 

In [6]:
### Load in .csv and convert to df

scraped_data = pd.read_csv('oakley_forum_data.csv')
scraped_data.dropna()

scraped_data

,title,selling_button,date,username
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,Trading Xmas player selection Sutro,NaN,1/3/24,BradyGod1569
4,NaN,NaN,NaN,NaN
...,...,...,...,...
1918,NaN,NaN,NaN,NaN
1919,Sold 2 M2 for sale,NaN,11/4/23,Titan X
1920,NaN,NaN,NaN,NaN
1921,NaN,NaN,NaN,Titan X
